# 차등 프라이버시 활용 방식 살펴보기

기계 학습 프로젝트에서는 대개 데이터 분석을 반복적으로 수행하여 데이터를 파악한 다음 예측 모델을 작성하는 데 가장 적합한 변수를 결정합니다. 데이터 분석 과정에서 일반적으로 사용되는 집계 및 통계 함수는 변수의 통계적 분포 및 상호 간 관계에 대한 인사이트를 제공합니다. 데이터의 양이 많을 때 집계 기능을 사용하면 데이터를 어느 정도 추상화할 수 있습니다. 반면 데이터의 양이 적거나 분석을 반복적으로 수행할 때는 집계된 결과에서도 개별 관찰 관련 세부 정보를 파악할 수 있습니다.

*차등 프라이버시*란 데이터에 "의미 없는 단어"를 추가하여 개별 데이터 요소의 "프라이버시"를 유지하는 기술입니다. 즉, 이 기술은 개별 값의 프라이버시를 유지하기에 충분한 의미 없는 단어를 추가하는 동시에 데이터의 전반적인 통계적 구성 일관성을 유지하고, 원래 원시 데이터에 집계 기능을 사용하는 경우에도 통계적으로 비슷한 결과가 생성되도록 하는 데 사용됩니다.

## SmartNoise SDK 설치

[*SmartNoise*](https://smartnoise.org/)는 OpenDP의 도구 키트입니다. OpenDP는 데이터 분석 및 기계 학습 프로젝트에서 차등 프라이버시 사용을 위한 구성 요소를 제공하기 위해 Microsoft와 하버드 대학교의 연구원, 그리고 기타 다양한 참가자들이 공동으로 진행하고 있는 프로젝트입니다.

> **참고**: SmartNoise는 현재 아직 개발 초기 단계 상태입니다.

먼저 SmartNoise Python SDK 패키지부터 설치합니다.

In [ ]:
!pip install opendp-smartnoise==0.1.2

## 데이터 로드

이제 원시 데이터를 살펴보겠습니다. 원시 데이터는 당뇨병 검사를 받은 환자의 레코드 10,000개가 포함된 데이터 세트입니다.

In [ ]:
import pandas as pd

data_path = 'data/diabetes.csv'
diabetes = pd.read_csv(data_path)
diabetes.describe()

위 코드의 출력에는 당뇨병 데이터 세트에 사용되는 변수의 주요 요약 통계가 표시됩니다.

## 분석 수행

SmartNoise를 사용하면 원본 데이터에 의미 없는 단어가 추가되는 분석을 작성할 수 있습니다. 의미 없는 단어가 추가되는 기본적인 수학 방식은 매우 복잡할 수 있습니다. 하지만 여기서는 SmartNoise가 대다수 세부 작업을 자동으로 처리합니다. 그러나 다음과 같은 몇 가지 개념은 알아 두면 유용합니다.

- **상한값 및 하한값**: *제한* 기능을 사용하여 변수 값의 상한값과 하한값을 설정합니다. SmartNoise에서 생성하는 의미 없는 단어가 원래 데이터의 예상 분포와 일치하도록 하려면 상한값과 하한값을 설정해야 합니다.
- **샘플 크기**: 특정 집계용으로 일관성 있는 차등 프라이빗 데이터를 생성하려면 SmartNoise가 생성할 데이터 샘플의 크기를 확인할 수 있어야 합니다.
- **엡실론**: *엡실론*이란 쉽게 설명하자면 데이터에 추가할 의미 없는 데이터 양의 역 측정값을 제공하는 음수가 아닌 값입니다. 엡실론 값이 작으면 데이터 세트의 프라이버시 수준이 높아지며, 엡실론 값이 크면 데이터 세트가 원래 데이터에 가까워집니다. 일반적으로는 0~1 사이의 엡실론 값을 사용해야 합니다. 엡실론과 또 다른 값인 *델타* 간에는 상관 관계가 설정됩니다. 델타는 분석에서 생성되는 보고서의 내용이 완전 프라이빗 데이터가 아닐 확률을 나타냅니다.

이러한 개념을 기억하면서 다음 코드를 살펴보고 실행해 보세요. 이 코드를 실행하면 차등 프라이빗 데이터에서 분석이 작성되며 평균 **Age** 값이 보고됩니다. 원래 원시 데이터의 실제 평균 값도 비교용으로 표시됩니다.

In [ ]:
import opendp.smartnoise.core as sn

cols = list(diabetes.columns)
age_range = [0.0, 120.0]
samples = len(diabetes)

with sn.Analysis() as analysis:
    # 데이터 로드
    data = sn.Dataset(path=data_path, column_names=cols)
    
    # 연령을 부동으로 변환
    age_dt = sn.to_float(data['Age'])
    
    # 연령 평균 가져오기
    age_mean = sn.dp_mean(data = age_dt,
                          privacy_usage = {'epsilon': .50},
                          data_lower = age_range[0],
                          data_upper = age_range[1],
                          data_rows = samples
                         )
    
analysis.release()

# 평균 연령의 비공개 추정치를 각기 다르게 인쇄
print("Private mean age:",age_mean.value)

# 실제 평균 연령 인쇄
print("Actual mean age:",diabetes.Age.mean())

## 히스토그램에서 데이터 분포 살펴보기

데이터를 분석하여 변수 분포를 파악할 때는 히스토그램이 흔히 사용됩니다.

일례로 당뇨병 데이터 세트의 실제 연령 분포를 살펴보겠습니다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

ages = list(range(0, 130, 10))
age = diabetes.Age

# 10년 기간의 저장소가 있는 히스토그램 그리기
n_age, bins, patches = plt.hist(age, bins=ages, color='blue', alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('True Age Distribution')
plt.show()
print(n_age.astype(int))

이번에는 Age의 차등 프라이빗 히스토그램을 비교용으로 확인해 보겠습니다.

In [ ]:
import matplotlib.pyplot as plt

with sn.Analysis() as analysis:
    data = sn.Dataset(path = data_path, column_names = cols)

    age_histogram = sn.dp_histogram(
            sn.to_int(data['Age'], lower=0, upper=120),
            edges = ages,
            upper = 10000,
            null_value = -1,
            privacy_usage = {'epsilon': 0.5}
        )
    
analysis.release()

plt.ylim([0,7000])
width=4
agecat_left = [x + width for x in ages]
agecat_right = [x + 2*width for x in ages]
plt.bar(list(range(0,120,10)), n_age, width=width, color='blue', alpha=0.7, label='True')
plt.bar(agecat_left, age_histogram.value, width=width, color='orange', alpha=0.7, label='Private')
plt.legend()
plt.title('Histogram of Age')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

print(age_histogram.value)

위에 나와 있는 것처럼 두 히스토그램은 매우 비슷하므로, 차등 프라이빗 데이터 기반 보고서도 원시 데이터 기반 보고서와 동일한 인사이트를 제공합니다.

## 공 분산 계산

변수 간의 관계를 설정하기 위해 분석을 수행하는 경우도 많습니다. SmartNoise에서는 이러한 관계를 쉽게 설정할 수 있도록 차등 프라이빗 *공 분산* 함수를 제공합니다.

In [ ]:
with sn.Analysis() as analysis:
    sn_data = sn.Dataset(path = data_path, column_names = cols)

    age_bp_cov_scalar = sn.dp_covariance(
                left = sn.to_float(sn_data['Age']),
                right = sn.to_float(sn_data['DiastolicBloodPressure']),
                privacy_usage = {'epsilon': 1.0},
                left_lower = 0.,
                left_upper = 120.,
                left_rows = 10000,
                right_lower = 0.,
                right_upper = 150.,
                right_rows = 10000)
analysis.release()
print('Differentially private covariance: {0}'.format(age_bp_cov_scalar.value[0][0]))
print('Actual covariance', diabetes.Age.cov(diabetes.DiastolicBloodPressure))

이 연습의 예제에서는 **Age**와 **DisatolicBloodPressure** 간의 공 분산이 '긍정'에 해당됩니다. 즉, 고령층 환자일수록 고혈압일 가능성이 높습니다.

## SQL 쿼리 사용

SmartNoise에서는 **분석** 기능을 사용할 수 있을 뿐 아니라, 데이터 원본을 대상으로 SQL 쿼리를 사용하여 차등 프라이빗 집계 결과를 검색할 수도 있습니다.

먼저 데이터 스키마 내 테이블의 메타데이터를 정의해야 합니다. **/metadata** 폴더의 **diabetes.yml** 파일과 같은 .yml 파일을 사용하면 메타데이터를 정의할 수 있습니다. 메타데이터는 테이블의 필드를 설명합니다. 가령 숫자 필드의 데이터 형식, 최소값과 최대값 등을 설명할 수 있습니다.

In [ ]:
from opendp.smartnoise.metadata import CollectionMetadata

meta = CollectionMetadata.from_file('metadata/diabetes.yml')
print (meta)

메타데이터를 정의한 후에는 쿼리 가능한 *판독기*를 만들 수 있습니다. 다음 예제에서는 Pandas 데이터 프레임에서 원시 데이터를 읽는 **PandasReader**, 그리고 **PandasReader**에 차등 프라이버시 레이어를 추가하는 **PrivateReader**를 만듭니다.

In [ ]:
from opendp.smartnoise.sql import PandasReader, PrivateReader

reader = PandasReader(diabetes, meta)
private_reader = PrivateReader(reader, meta)
print('Readers ready.')

이제 프라이빗 판독기로 집계 결과 세트를 반환하는 SQL 쿼리를 제출할 수 있습니다.

In [ ]:
query = 'SELECT Diabetic, AVG(Age) AS AvgAge FROM diabetes.diabetes GROUP BY Diabetic'

result_dp = private_reader.execute(query)
print(result_dp)

해당 결과를 원시 데이터의 동일 집계 결과와 비교해 보겠습니다.

In [ ]:
result = reader.execute(query)
print(result)

명시적 엡실론 값을 지정하여 **PrivateReader**의 동작을 사용자 지정할 수 있습니다.

그러면 엡실론 값이 큰(프라이버시 수준 낮음) 판독기와 엡실론 값이 작은(프라이버시 수준 높음) 판독기를 각각 실행해 보겠습니다.

In [ ]:
low_privacy_reader = PrivateReader(reader, meta, 5.0)  # 대규모 엡실론, 개인정보 보호 약화
result = low_privacy_reader.execute(query)
print(result)
print()

high_privacy_reader = PrivateReader(reader, meta, 0.1)  # 소규모 엡실론, 개인정보 보호 강화
result = high_privacy_reader.execute(query)
print(result)

엡실론 값이 큰(프라이버시 수준 낮음) 판독기의 결과가 엡실론 값이 작은(프라이버시 수준 높음) 판독기의 결과에 비해 원시 데이터의 실제 결과와 더 비슷합니다.

## 더 알아보기

SmartNoise를 사용해 차등 프라이버시를 적용하는 방법에 대해 자세히 알아보려면 [https://smartnoise.org](https://smartnoise.org/)를 참조하세요.